In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, ZeroPadding2D, Conv2DTranspose, UpSampling2D
from keras.layers.merge import concatenate, add, average
from keras.layers import LeakyReLU, BatchNormalization, Activation, Dropout, MaxPooling2D
from keras import regularizers


In [3]:
import numpy as np
import pandas as pd

import zipfile
import glob
import torch
import matplotlib.pylab as plt
import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, concatenate, Input
from tensorflow.keras import Model

import warnings
warnings.filterwarnings("ignore")

In [4]:
#GPU 사용 가능 여부 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [4]:
tf.test.is_gpu_available()

In [4]:
#data load
train_files = glob.glob('../input/radar-image/train/train/*.npy')
len(train_files)

In [5]:
#train, test 데이터 분할

from sklearn.model_selection import train_test_split

train, test = train_test_split(train_files, test_size = 0.2, shuffle=True, random_state=1004)

In [6]:
print(len(test))
print(len(train))

In [7]:
#data check
data_1st = np.load(train[50])
data_1st.shape

In [8]:
#train data visualization

#color map setting
color_map = plt.cm.get_cmap('RdBu')
color_map = color_map.reversed()

#visualization
plt.style.use('fivethirtyeight')
plt.figure(figsize = (20,20))

for i in range(4):
    plt.subplot(1,5,i+1)
    plt.imshow(data_1st[:,:,i],cmap = color_map)
    
plt.subplot(1,5,5)
plt.imshow(data_1st[:,:,-1], cmap=color_map)
plt.show()

In [ ]:
#histogram equalization visualization
'''
color_map = plt.cm.get_cmap('RdBu')
color_map = color_map.reversed()

#원래 이미지
img = data_1st[:,:,1]
#히스토그램 균일화 이미지
equ = cv2.equalizeHist(img)

#img와 equ를 imshow
dst = np.hstack((img, equ))
plt.imshow(dst, clim = [0,255], cmap = color_map)
#cv2.imshow('equ', dst)
#cv2.waitKey()
#cv2.destroyAllWindows()

'''

In [ ]:
#Intensity Range Standardization
from medpy.filter import IntensityRangeStandardization

color_map = plt.cm.get_cmap('RdBu')
color_map = color_map.reversed()

#원래 이미지
img = data_1st[:,:,1]
#명암표준화
irs = IntensityRangeStandardization()
irs_model, irs_image = irs.train_transform(img)

#img와 irs_image를 imshow
dst = np.hstack((img, irs_image))
plt.imshow(dst, clim = [0,255], cmap = color_map)
#cv2.imshow('equ', dst)
#cv2.waitKey()
#cv2.destroyAllWindows()


In [ ]:
#이미지 정규화도 할것.

In [ ]:
#히스토그램 표준화 -> 성능이 안좋아짐.
        #for j in range(5):
        #    dataset[:,:,j] = cv2.equalizeHist(dataset[:,:,j])

In [8]:
#data preprocessing

def trainGenerator():
    for file in train_files:
        dataset = np.load(file)
        
        target = dataset[:,:,-1].reshape(120,120,1)
        remove_minus = np.where(target<0,0,target)
        feature = dataset[:,:,:4]
        
        yield (feature, remove_minus)

def valGenerator():
    for file in val_files:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(120,120,1)        
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4]


        yield (feature, remove_minus)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))
train_dataset = train_dataset.batch(4).prefetch(1)

In [ ]:
'''
#unet model
def deepUnet_model(input_layer, start_neurons):
    
    #unet 1layer
    conv1 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(input_layer)
    bn1 = BatchNormalization()(conv1)
    conv1_2 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding = 'same')(bn1)
    bn1_2 = BatchNormalization()(conv1_2)  #(120,120,32)
    
    #shortcut part  #(120,120,32)
    shortcut1 = Conv2D(start_neurons *1 ,(1,1), activation='relu',padding='same')(input_layer)
    
    add1 = add([bn1_2, shortcut1]) #(120,120,64)
    add1 = Activation(activation = 'relu')(add1)
    pool1 = MaxPooling2D((2, 2))(add1)  #(60,60,64)

    
    #encoder 2layer  (60,60,64)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    bn2 = BatchNormalization()(conv2)
    conv2_2 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding = 'same')(bn2)
    bn2_2 = BatchNormalization()(conv2_2)  #(60,60,32)
    
    #shortcut part  (60,60,32)
    shortcut2 = Conv2D(start_neurons *1 ,(1,1), activation='relu',padding='same')(pool1)
    
    add2 = add([bn2_2, shortcut2])  #(60,60,64)
    add2 = Activation(activation = 'relu')(add2)
    pool2 = MaxPooling2D((2, 2))(add2)  #(30,30,64)

    convm = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(add2)
    #(30,30,64)
    
    
    #decoder1
    #(60,60,64)
    uconv2 = concatenate([convm, conv2])
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(deconv2)
    uconv2 = BatchNormalization()(uconv2)

    
    #decoder2
    uconv1 = concatenate([uconv2, conv1])
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
   
    return output_layer

    
input_layer = Input((120, 120, 4))
output_layer = deepUnet_model(input_layer,32)

'''

In [9]:
#unet model
def deepUnet_model(input_layer, start_neurons):
    
    #unet 1layer
    conv1 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(input_layer)
    bn1 = BatchNormalization()(conv1)
    conv1_2 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding = 'same')(bn1)
    bn1_2 = BatchNormalization()(conv1_2)  #(120,120,32)
    
    #shortcut part  #(120,120,32)
    shortcut1 = Conv2D(start_neurons *1 ,(1,1), activation='relu',padding='same')(input_layer)
    
    add1 = add([bn1_2, shortcut1]) #(120,120,64)
    add1 = Activation(activation = 'relu')(add1)
    pool1 = MaxPooling2D((2, 2))(add1)  #(60,60,64)

    
    
    #encoder 2layer  (60,60,64)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    bn2 = BatchNormalization()(conv2)
    conv2_2 = Conv2D(start_neurons * 1, (3, 3), activation='relu', padding = 'same')(bn2)
    bn2_2 = BatchNormalization()(conv2_2)  #(60,60,32)
    
    #shortcut part  (60,60,32)
    shortcut2 = Conv2D(start_neurons *1 ,(1,1), activation='relu',padding='same')(pool1)
    
    add2 = add([bn2_2, shortcut2])  #(60,60,64)
    add2 = Activation(activation = 'relu')(add2)
    pool2 = MaxPooling2D((2, 2))(add2)  #(30,30,64)

    
    
    convm = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool2)
    convm = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    #(30,30,64)
    #convm을 uppooling해서 처리
    
    
    #decoder1
    #(60,60,64)
    uconv2 = concatenate([convm, conv2])
    
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    #shortcut
    shortcut_d2 = Conv2D(start_neurons *1 ,(1,1), activation='relu',padding='same')(uconv2)
    #add
    add_d2 = add([uconv2, shortcut_d2])
    add_d2 = Activation(activation = 'relu')(add_d2)
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(add_d2)
    

   

    #decoder2
    uconv1 = concatenate([deconv2, conv1])
    
    uconv1 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    #shortcut
    shortcut_d1 = Conv2D(start_neurons *1 ,(1,1), activation='relu',padding='same')(uconv1)
    #add
    add_d1 = add([uconv1, shortcut_d1])
    add_d1 = Activation(activation = 'relu')(add_d1)
    #deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(add_d1)
    
    
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(add_d1)
   
    return output_layer

    
input_layer = Input((120, 120, 4))
output_layer = deepUnet_model(input_layer,32)

In [ ]:
with tf.device("/device:GPU:0"):
    model = Model(input_layer, output_layer)
    model.compile(loss='mae', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
#gpu 사용하여 학습해보자
with tf.device("/device:GPU:0"):
    model.fit(train_dataset, epochs = 5, verbose=1)

In [ ]:
len(train)

In [ ]:
kf = KFold(n_splits = 5, random_state = 42)
for t, v in kf.split(train):
        print(type(v))
        print(v)
        print(t)
        train_files = train[t]
        val_files = train[v]

In [11]:
train = np.array(train)

In [12]:
import os
root_path = './'

In [13]:
#K-fold 해보기
import gc
from sklearn.model_selection import KFold

all_scores = []
all_preds = []
idx = 0

kf = KFold(n_splits = 5, random_state = 42)
with tf.device("/device:GPU:0"):
    for fold, (t, v) in enumerate(kf.split(train)):
        print(type(v))
        train_files = train[t]
        val_files = train[v]
        
        #데이터 전처리
        train_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))
        train_dataset = train_dataset.batch(4).prefetch(1)
        
        val_dataset = tf.data.Dataset.from_generator(valGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))
        val_dataset = val_dataset.batch(4).prefetch(1)
        
        
        #데이터 정리
        input_layer = Input((120,120,4))
        output_layer = deepUnet_model(input_layer,32)
        
        
        model = Model(input_layer, output_layer)
        adam = tf.keras.optimizers.Adam()
        model.compile(optimizer=adam, loss='mae')

        
        #epoch마다 callback함수 실행
        #모델의 개선이 없을 경우 learning rate를 조절해 모델의 개선을 유도. 
        callbacks = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.2, patience=2, verbose=0, mode='min',
                min_delta=0.0001, cooldown=0, min_lr=0)

        sv = tf.keras.callbacks.ModelCheckpoint(
            os.path.join(root_path,f'{fold}-rain.h5'),
            monitor='val_loss', verbose=0, save_best_only=True,
            save_weights_only=True, mode='min', save_freq='epoch')
        
        #epoch수를 좀 키울 필요가 있을듯
        #earlystopping  = EarlyStopping(monitor = 'val_loss', patience = 10)
        
        model.fit(train_dataset, epochs = 1, verbose=1, validation_data=val_dataset, callbacks=[callbacks, sv])

        del model
        gc.collect()
        
        
        '''
        #모델 생성
        model = Model(input_layer, output_layer)
        model.compile(loss='mae', optimizer='adam')
        
        callbacks = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.2, patience=2, verbose=0, mode='min',
            min_delta=0.0001, cooldown=0, min_lr=0
        )
        
        sv = tf.keras.callbacks.ModelCheckpoint(
            os.path.join(root_path,f'{fold}-rain.h5'), monitor='val_loss', verbose=0, save_best_only=True,
            save_weights_only=True, mode='min', save_freq='epoch')

        model.fit(train_dataset, epochs = 1, verbose=1, validation_data=val_dataset, callbacks=[callbacks, sv])

        del model
        gc.collect()
        
        
        #모델 학습
        history = model.fit(train_dataset, epochs = 3, verbose=1, validation_data = val_dataset)
        
        #모델 저장
        model.save('./deepUnet_'+str(idx)+'_lastEpoch.h5')
        idx += 1
        
        #시각화
        plt.plot(history.history['loss'], label = 'train'+strd(idx))
        
        #scores = model.evaluate(val_dataset)
        #all_scores.append(scores)
        
        #pred = model.predict(x_test)
        #all_preds.append(pred)
        '''

In [14]:
#학습된 모델 불러오기
input_layer = Input((120,120,4))
output_layer = deepUnet_model(input_layer,32)
model = Model(input_layer, output_layer)

In [15]:
#테스트 데이터 로드
x_test = []
for file in tqdm(test, desc = 'test_x'):
    data = np.load(file)
    feature = data[:,:,:4]
    x_test.append(feature)
    
x_test = np.array(x_test)



y_test = []
for file in tqdm(test, desc = 'test_y'):
    data = np.load(file)
    target = data[:,:,-1].reshape(120,120,1)
    remove_minus = np.where(target<0,0,target)
    y_test.append(remove_minus)
    
y_test = np.array(y_test)

In [17]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=42)
total_val = []
total_answer = []
test_preds = []

for fold, (train, val) in enumerate(kf.split(train)):

    val_file = train_files[val]
    
    X_val = []
    X_answers = []
  
    for file in tqdm(val_file, desc='val'):
        data = np.load(file)
        X_answer = data[:,:,-1]
        X_input = data[:,:,:4]
  
        X_val.append(X_input)
        X_answers.append(X_answer)
    
    X_val = np.array(X_val)
    X_answers = np.array(X_answers)
    model.load_weights(os.path.join(root_path,f'{fold}-rain.h5'))

    val_pred = model.predict(X_val)
    test_pred = model.predict(x_test)

    #test_pred = test_pred.reshape(-1,14400)
    #final_X_val = val_pred.reshape(-1,14400)
    total_val.append(val_pred)
    total_answer.append(X_answers)
    test_preds.append(test_pred)
  
    del val_pred, X_val, X_answers
    gc.collect()

In [1]:
len(test_preds[0])

In [24]:
print(MAE(y_test, test_preds[0]))
print(MAE(y_test, test_preds[1]))
print(MAE(y_test, test_preds[2]))
print(MAE(y_test, test_preds[3]))
print(MAE(y_test, test_preds[4]))

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
#unet model
def unet_model(input_layer, start_neurons):
    
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)

    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
   
    return output_layer

    
in_layer = Input((120, 120, 4))
out_layer = unet_model(in_layer,64)

In [ ]:
with tf.device("/device:GPU:0"):
    model2 = Model(in_layer, out_layer)
    model2.compile(loss='mae', optimizer='adam')

In [ ]:
model2.summary()

In [ ]:
#gpu 사용하여 학습해보자
with tf.device("/device:GPU:0"):
    model2.fit(train_dataset, epochs = 5, verbose=1)

In [ ]:
'''
#test 데이터 만들기
import random
#test_files = random.sample(train_files, int(len(train_files)*0.3))
test_files = random.sample(train_files, 2674)
'''

In [ ]:
len(test_files)

In [ ]:
x_test = []
for file in tqdm(test, desc = 'test'):
    data = np.load(file)
    feature = data[:,:,:4]
    x_test.append(feature)
    
x_test = np.array(x_test)

In [ ]:
y_test = []
for file in tqdm(test, desc = 'test'):
    data = np.load(file)
    target = data[:,:,-1].reshape(120,120,1)
    remove_minus = np.where(target<0,0,target)
    y_test.append(remove_minus)
    
y_test = np.array(y_test)

In [ ]:
#모델 성능 확인하기
pred1 = model.predict(x_test)

In [ ]:
#모델 성능 확인하기
pred2 = model2.predict(x_test)

In [22]:
#test data mae 측정
def MAE(y, pred):
    return np.mean(np.abs(y-pred))

In [ ]:
print(MAE(y_test, pred1))

In [ ]:
print(MAE(y_test, pred2))

In [ ]:
#시각화
#predict visualization

#color map setting
color_map = plt.cm.get_cmap('RdBu')
color_map = color_map.reversed()

#visualization
plt.style.use('fivethirtyeight')
plt.figure(figsize = (20,20))

plt.subplot(1,5,1)
plt.imshow(y_test[6], cmap = color_map)
plt.subplot(1,5,2)
#plt.imshow(pred2[2], cmap=color_map)
#plt.subplot(1,5,3)
plt.imshow(pred1[6], cmap = color_map)
#plt.subplot(1,5,4)
#plt.imshow(pred3[2], cmap=color_map)

plt.show()